In [3]:
import os, sys
from os.path import join

import nibabel as nib
import numpy as np
import pandas as pd

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Make timing files for each run

   - each run was composed of 6 blocks. Each block began with 20s of rest, and then 10s of task (either motor squeezing or imagery)
   - there was 6 seconds of disdaqs at the beginning
   - However, the data collection started 2 TRs AFTER the task started. Thus with a TR of 1s, there were 2 seconds of task before the task started. Likewise, for a TR of 2s, there was 4 seconds of task before the task started. This was due to the 2 dummy scans that Philips inserts at the beginning of the each scan

In [4]:
trialStarts = []
currentTime = 0

for b in range(6):
    # start time for rest
    trialStarts.append(currentTime)

    # increment by rest duration
    currentTime += 20
    
    # start time for task
    trialStarts.append(currentTime)
    
    # increment by the task duration
    currentTime += 10

# add in offset for 
trialStarts = np.array(trialStarts) + 6

In [5]:
## make separate arrays for TR=1s and TR=2s
# subtract 2 dummy scans (2s)
onsets_1sTR = trialStarts - 2

# subtract 2 dummy scans (4s)
onsets_2sTR = trialStarts - 4

In [6]:
# hardcoded order on each run
trialTypes = ['rest', 'motor', 
             'rest', 'imagery',
             'rest', 'imagery',
             'rest', 'motor',
             'rest', 'imagery',
             'rest', 'motor']


In [7]:
# make a dataframe
trialOnsets = pd.DataFrame({'trialTypes': trialTypes, 'onsets_TR1': onsets_1sTR, 'onsets_TR2': onsets_2sTR, 'weights': np.ones(onsets_1sTR.shape)})

In [8]:
def trialDuration(row):
    if row['trialTypes'] == 'rest':
        val = 20
    else:
        val = 10
    return val
trialOnsets['duration'] = trialOnsets.apply(trialDuration, axis=1)

In [9]:
trialOnsets

,onsets_TR1,onsets_TR2,trialTypes,weights,duration
0,4,2,rest,1.0,20
1,24,22,motor,1.0,10
2,34,32,rest,1.0,20
3,54,52,imagery,1.0,10
4,64,62,rest,1.0,20
5,84,82,imagery,1.0,10
6,94,92,rest,1.0,20
7,114,112,motor,1.0,10
8,124,122,rest,1.0,20
9,144,142,imagery,1.0,10


### Write out FSL 3col files

In [63]:
# 1s TR files
for trialType in ['rest', 'motor', 'imagery']:
    df = trialOnsets.loc[trialOnsets.trialTypes == trialType, ['onsets_TR1','duration', 'weights']]
    df.to_csv('../data/subject001/timingFiles/TR1_{}.txt'.format(trialType),
              sep='\t', 
              header=False,
              index=False)
    
# 2s TR files
for trialType in ['rest', 'motor', 'imagery']:
    df = trialOnsets.loc[trialOnsets.trialTypes == trialType, ['onsets_TR2','duration', 'weights']]
    df.to_csv('../data/subject001/timingFiles/TR2_{}.txt'.format(trialType),
              sep='\t', 
              header=False,
              index=False)

# Prep Masks

For localizer-based masks, we analyzed the first run (pyneal_002) and computed constrast stats for `motor > rest` and `imagery > rest`. For each contrast, we took the coordinates of the voxel with the max zstat:

* **motor > rest**: (28, 28, 10)
* **imagery > rest**: (34, 35, 14)

Since each run has different slice prescriptions and coverage, we need to convert these coordinates to MNI space, draw a 5mm sphere around the point, and save as a new mask.

Then, for each run, transform the MNI-space mask down to subject functional space for that run. Mask by the whole brain mask to make sure no non-brain voxels are incldued in the final mask

### Convert peak coordinate for each contrast to MNI space

In [23]:
# motor > rest peak coord in MNI space
!echo 28 28 10 | img2imgcoord -src ../data/subject001/pyneal_002/exampleFunc.nii.gz -dest /usr/local/fsl/data/standard/MNI152_T1_1mm_brain.nii.gz -xfm ../data/subject001/pyneal_002/mask_transforms/func2mni.mat

Coordinates in Destination volume (in voxels)
126.749  90.0093  133.427


In [25]:
# imagery > rest peak coord in MNI space
!echo 34 35 14 | img2imgcoord -src ../data/subject001/pyneal_002/exampleFunc.nii.gz -dest /usr/local/fsl/data/standard/MNI152_T1_1mm_brain.nii.gz -xfm ../data/subject001/pyneal_002/mask_transforms/func2mni.mat

Coordinates in Destination volume (in voxels)
108.378  114.904  145.219


### Make 5mm sphere mask in MNI space around each peak

In [30]:
# motor > rest
!fslmaths /usr/local/fsl/data/standard/MNI152_T1_1mm_brain.nii.gz -mul 0 -add 1 -roi 127 1 90 1 133 1 0 1 tmpMotorPoint_MNI -odt float

!fslmaths tmpMotorPoint_MNI.nii.gz -kernel sphere 5 -fmean -bin ../data/subject001/masks/motorSphere_5mm_MNI -odt float

!rm tmpMotorPoint_MNI.nii.gz

In [31]:
# imagery > rest
!fslmaths /usr/local/fsl/data/standard/MNI152_T1_1mm_brain.nii.gz -mul 0 -add 1 -roi 108 1 114 1 145 1 0 1 tmpImageryPoint_MNI -odt float

!fslmaths tmpImageryPoint_MNI.nii.gz -kernel sphere 5 -fmean -bin ../data/subject001/masks/imagerySphere_5mm_MNI -odt float

!rm tmpImageryPoint_MNI.nii.gz